In [37]:
import tensorflow as tf
import numpy as np
import argparse
import socket
import importlib
import os
import imageio
#import scipy.misc
import sys

In [38]:
BASE_DIR = os.getcwd()
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
import provider
from utils import pc_util
from tf_util import log_string

In [39]:
BATCH_SIZE = 4
NUM_POINT = 1024
MODEL_PATH = 'log/model.ckpt'
GPU_INDEX = 0
MODEL = importlib.import_module('dgcnn')  # import network module
DUMP_DIR = 'dump'
VISU = False

In [40]:
if not os.path.exists(DUMP_DIR): os.mkdir(DUMP_DIR)
LOG_FOUT = open(os.path.join(DUMP_DIR, 'log_evaluate.txt'), 'w')
LOG_FOUT.write("BATCH_SIZE = 4, NUM_POINT = 1024, GPU_INDEX = 0\n")

NUM_CLASSES = 40
SHAPE_NAMES = [line.rstrip() for line in \
    open(os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/shape_names.txt'))] 

HOSTNAME = socket.gethostname()

In [41]:
# ModelNet40 test files
TEST_FILES = provider.get_data_files(\
    os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/test_files.txt'))

In [42]:
def evaluate(num_votes):
    with tf.device('/gpu:'+str(GPU_INDEX)):
        pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
        is_training_pl = tf.placeholder(tf.bool, shape=())

        # simple model
        pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)
        loss = MODEL.get_loss(pred, labels_pl, end_points)
        
        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()
        
    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = True
    sess = tf.Session(config=config)

    # Restore variables from disk.
    saver.restore(sess, MODEL_PATH)
    log_string(LOG_FOUT, "Model restored.")

    ops = {'pointclouds_pl': pointclouds_pl,
           'labels_pl': labels_pl,
           'is_training_pl': is_training_pl,
           'pred': pred,
           'loss': loss}

    eval_one_epoch(sess, ops, num_votes)

In [43]:
def eval_one_epoch(sess, ops, num_votes=1, topk=1):
    error_cnt = 0
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    fout = open(os.path.join(DUMP_DIR, 'pred_label.txt'), 'w')
    for fn in range(len(TEST_FILES)):
        log_string(LOG_FOUT, '----'+str(fn)+'----')
        current_data, current_label = provider.load_h5(TEST_FILES[fn])
        current_data = current_data[:,0:NUM_POINT,:]
        current_label = np.squeeze(current_label)
        
        file_size = current_data.shape[0]
        num_batches = file_size // BATCH_SIZE
        
        for batch_idx in range(num_batches):
            start_idx = batch_idx * BATCH_SIZE
            end_idx = (batch_idx+1) * BATCH_SIZE
            cur_batch_size = end_idx - start_idx
            
            # Aggregating START
            batch_loss_sum = 0  # sum of losses for the batch
            batch_pred_sum = np.zeros((cur_batch_size, NUM_CLASSES))  # predictions of classes for a batch of different clouds are summed up along different rotation angle
            # batch_pred_classes = np.zeros((cur_batch_size, NUM_CLASSES))  # batch of one-hot predictions of classes
            for vote_idx in range(num_votes):
                rotated_data = provider.rotate_point_cloud_by_angle(current_data[start_idx:end_idx, :, :],
                                                  vote_idx/float(num_votes) * np.pi * 2)
                feed_dict = {ops['pointclouds_pl']: rotated_data,
                             ops['labels_pl']: current_label[start_idx:end_idx],
                             ops['is_training_pl']: is_training}
                loss_val, pred_val = sess.run([ops['loss'], ops['pred']],
                                          feed_dict=feed_dict)
                batch_pred_sum += pred_val  # add predictions for current batch based on current angle to a sum
                batch_loss_sum += (loss_val * cur_batch_size / float(num_votes))  # add loss value for current batch based on current angle to a sum
            pred_val = np.argmax(batch_pred_sum, 1)  # class with maximal number of votes along different angles
            # Aggregating END
            
            correct = np.sum(pred_val == current_label[start_idx:end_idx])
            total_correct += correct
            total_seen += cur_batch_size
            loss_sum += batch_loss_sum

            for i in range(start_idx, end_idx):  # for every cloud of current batch
                l = current_label[i]
                total_seen_class[l] += 1  # count total seen and total correct
                total_correct_class[l] += (pred_val[i-start_idx] == l)
                fout.write('%d, %d\n' % (pred_val[i-start_idx], l))

                # if prediction is wrong, save the three-view image as jpg
                if pred_val[i-start_idx] != l and VISU: # ERROR CASE, DUMP!
                    img_filename = '%d_label_%s_pred_%s.jpg' % (error_cnt, SHAPE_NAMES[l],
                                                           SHAPE_NAMES[pred_val[i-start_idx]])
                    img_filename = os.path.join(DUMP_DIR, img_filename)
                    output_img = pc_util.point_cloud_three_views(np.squeeze(current_data[i, :, :]))
                    #scipy.misc.imsave(img_filename, output_img)
                    imageio.imwrite(img_filename, output_img)
                    error_cnt += 1
                
    log_string(LOG_FOUT, 'eval mean loss: %f' % (loss_sum / float(total_seen)))
    log_string(LOG_FOUT, 'eval accuracy: %f' % (total_correct / float(total_seen)))
    log_string(LOG_FOUT, 'eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))

    # log accuracy of every class prediction
    class_accuracies = np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float)
    for i, name in enumerate(SHAPE_NAMES):
        log_string(LOG_FOUT, '%10s:\t%0.3f' % (name, class_accuracies[i]))

In [44]:
if __name__=='__main__':
    with tf.Graph().as_default():
        evaluate(num_votes=12)
    LOG_FOUT.close()

INFO:tensorflow:Restoring parameters from log/model.ckpt
Model restored.
----0----
----1----
eval mean loss: 1.453585
eval accuracy: 0.907212
eval avg class acc: 0.879930
  airplane:	1.000
   bathtub:	0.960
       bed:	0.990
     bench:	0.700
 bookshelf:	0.950
    bottle:	0.980
      bowl:	1.000
       car:	0.990
     chair:	0.960
      cone:	0.950
       cup:	0.750
   curtain:	0.900
      desk:	0.802
      door:	0.900
   dresser:	0.744
flower_pot:	0.300
 glass_box:	0.930
    guitar:	1.000
  keyboard:	1.000
      lamp:	0.900
    laptop:	1.000
    mantel:	0.950
   monitor:	0.990
night_stand:	0.791
    person:	0.950
     piano:	0.950
     plant:	0.780
     radio:	0.950
range_hood:	0.960
      sink:	0.800
      sofa:	0.980
    stairs:	0.900
     stool:	0.800
     table:	0.820
      tent:	0.950
    toilet:	0.990
  tv_stand:	0.850
      vase:	0.830
  wardrobe:	0.650
      xbox:	0.600
